In [1]:
import pandas as pd
from bertopic import BERTopic
import random
from bertopic.representation import KeyBERTInspired
import numpy as np
import bertopic
from sklearn.metrics.pairwise import cosine_similarity

import scipy
print(scipy.__version__)



print(np.__version__)
print(pd.__version__)
print(bertopic.__version__)


1.9.2
1.23.4
1.5.1
0.15.0


In [2]:
from platform import python_version

print(python_version())

3.9.13


In [3]:
%pip install openpyxl


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
parl_model = BERTopic.load("ParlModel")
twitter_model = BERTopic.load("tweet_model")


twitter_model_info = twitter_model.get_topic_info()
parl_model_info = parl_model.get_topic_info()


In [5]:
twitter_model_info.to_excel('../data/twitter_model_info.xlsx')
parl_model_info.to_excel('../data/parl_model_info.xlsx')

In [6]:
sim_matrix = cosine_similarity(twitter_model.topic_embeddings_, parl_model.topic_embeddings_)

In [7]:
print(twitter_model_info.shape)
print(parl_model_info.shape)

print(type(sim_matrix))
with open('sim_matrix.npy', 'wb') as f:

    np.save(f, sim_matrix)


(99, 5)
(133, 5)
<class 'numpy.ndarray'>


In [8]:
parl_model_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9129,-1_präsident_gesetzentwurf_gesagt_weil,"[präsident, gesetzentwurf, gesagt, weil, bunde...","[Herr Präsident! Meine Damen und Herren! 2,4 M..."
1,0,563,0_epidemischen_epidemische_coronakrise_coronap...,"[epidemischen, epidemische, coronakrise, coron...",[Herr Präsident! Meine Damen! Meine Herren! No...
2,1,534,1_präsidentin_minister_bedanken_diskutieren,"[präsidentin, minister, bedanken, diskutieren,...",[Frau Präsidentin! Meine lieben Kolleginnen un...
3,2,461,2_politik_demokratie_partei_parlament,"[politik, demokratie, partei, parlament, debat...",[Sehr geehrter Herr Präsident! Liebe Kolleginn...
4,3,422,3_migrationspolitik_migrationspakt_einwanderun...,"[migrationspolitik, migrationspakt, einwanderu...","[Frau Kollegin Brugger, da wollten Sie Herrn D..."
...,...,...,...,...,...
128,127,13,127_aserbaidschanische_aserbaidschan_armenien_...,"[aserbaidschanische, aserbaidschan, armenien, ...",[Vielen Dank. – Herr Präsident! Meine Damen un...
129,128,12,128_krisen_krise_bündnispolitisch_krisenregelu...,"[krisen, krise, bündnispolitisch, krisenregelu...",[Herr Präsident! Liebe Kolleginnen und Kollege...
130,129,12,129_verfassungsadäquates_verfassungsadäquat_ve...,"[verfassungsadäquates, verfassungsadäquat, ver...",[Herr Präsident! Meine sehr geehrten Damen und...
131,130,12,130_rechtsextremismus_linksextremismus_extremi...,"[rechtsextremismus, linksextremismus, extremis...",[Hochverehrter Herr Präsident! Meine Damen und...


In [9]:
out_scores = []
out_debate_topics = []
out_debate_index = []

twitter_1_index = []
twitter_2_index = []
twitter_3_index = []

twitter_1_topics = []
twitter_2_topics = []
twitter_3_topics = []


pairs = []

print(sim_matrix.shape)





for row_ind in range(sim_matrix.shape[1]):
    row = sim_matrix[:,row_ind]

    pairs.append((row_ind, np.argmax(row)))

    out_scores.append(sim_matrix[np.argmax(row), row_ind])
    
    out_debate_index.append(parl_model_info.iloc[row_ind]['Topic'])
    out_debate_topics.append(parl_model_info.iloc[row_ind]['Representation'])

    
    # get the 3 arguments of the largest 3 values
    sorted_indices = np.argsort(row)[::-1]
    
    # Take the first three indices
    largest_indices = sorted_indices[:3]

    twitter_1_index.append(twitter_model_info.iloc[largest_indices[0]]['Topic'])
    twitter_1_topics.append(twitter_model_info.iloc[largest_indices[0]]['Representation'])

    twitter_2_index.append(twitter_model_info.iloc[largest_indices[1]]['Topic'])
    twitter_2_topics.append(twitter_model_info.iloc[largest_indices[1]]['Representation'])

    twitter_3_index.append(twitter_model_info.iloc[largest_indices[2]]['Topic'])
    twitter_3_topics.append(twitter_model_info.iloc[largest_indices[2]]['Representation'])

    

df = pd.DataFrame({
    'debate_topic': out_debate_topics, 
    'debate_index': out_debate_index, 
    'twitter_1_topic': twitter_1_topics,
    'twitter_1_index': twitter_1_index,
    'twitter_2_topic': twitter_2_topics,
    'twitter_2_index': twitter_2_index,
    'twitter_3_topic': twitter_3_topics,
    'twitter_3_index': twitter_3_index,
    'score': out_scores})
df.to_csv('topic_mapping_02.csv', index=False)


(99, 133)
[(0, 1), (1, 8), (2, 1), (3, 1), (4, 39), (5, 7), (6, 10), (7, 12), (8, 23), (9, 1), (10, 9), (11, 72), (12, 11), (13, 9), (14, 7), (15, 16), (16, 5), (17, 73), (18, 51), (19, 55), (20, 21), (21, 7), (22, 38), (23, 44), (24, 79), (25, 28), (26, 14), (27, 35), (28, 79), (29, 42), (30, 5), (31, 26), (32, 30), (33, 76), (34, 10), (35, 69), (36, 86), (37, 5), (38, 81), (39, 70), (40, 38), (41, 1), (42, 98), (43, 56), (44, 51), (45, 65), (46, 7), (47, 5), (48, 7), (49, 1), (50, 12), (51, 48), (52, 29), (53, 16), (54, 37), (55, 81), (56, 34), (57, 81), (58, 51), (59, 46), (60, 70), (61, 48), (62, 92), (63, 55), (64, 74), (65, 1), (66, 78), (67, 40), (68, 66), (69, 1), (70, 56), (71, 1), (72, 1), (73, 73), (74, 30), (75, 5), (76, 36), (77, 76), (78, 30), (79, 54), (80, 72), (81, 98), (82, 81), (83, 16), (84, 51), (85, 74), (86, 47), (87, 24), (88, 81), (89, 9), (90, 7), (91, 1), (92, 98), (93, 48), (94, 92), (95, 55), (96, 12), (97, 1), (98, 81), (99, 92), (100, 16), (101, 81), (102